In [1]:
##########==========##########==========##########==========##########==========

In [ ]:
print('TODO: PPTX13 code is SLOPPY quick code; will need to be rewritten ASAP')

## HEAD

#### HEAD01 - load libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import matplotlib

#### HEAD02 - load data files

In [3]:
def load_pickle(addr):
    conn = open(addr, 'rb')
    the_object = pickle.load(conn)
    conn.close()
    return the_object

handle_data = pd.read_excel('A_Input/twitter_handles.xlsx')
user_data = pd.read_csv('B_Process/user_data.csv')
tweet_data = pd.read_csv('B_Process/tweet_data.csv')

best_model_stats = load_pickle('B_Process/best_model_stats.pickle')

/Users/s8/opt/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## PPTX

#### PPTX08 - Data Sources and Collection (Slide 10) - Calculate data collection statistics

In [4]:
## calculate data collection statistics
def calculate_collection_stats(
    hd = handle_data, ud = user_data, td = tweet_data):
    
    ## define useful indices
    congress_idx = ['USA House', 'USA Senate']
    
    ## calculate handle_data statistics
    hd_stats = {
        'Total': hd.shape[0],
        'Congress': hd.group.value_counts().loc[congress_idx].sum(),
        'Republican': hd.party.value_counts().loc['Republican']
    }
    
    ## calculate user statistics
    include_idx = ud.ml_set != 'exclude'
    ud_stats = {
        'Total': ud.loc[include_idx].shape[0],
        'Congress': ud.group.loc[include_idx].value_counts().loc[
            congress_idx].sum(),
        'Republican': ud.party.loc[include_idx].value_counts().loc['Republican']
    }
    
    ##calcualte tweet statistics
    td_stats = {
        'Total': td.shape[0],
        'Congress': td.merge(ud, how = 'left', right_on = 'handle',
            left_on = 'screen_name').group.value_counts().loc[congress_idx].sum(),
        'Republican': td.merge(ud, how = 'left', right_on = 'handle',
            left_on = 'screen_name').party.value_counts().loc['Republican'].sum()
    }
    
    ## compile statistics
    col_stats = {'Handles': hd_stats, 'Users': ud_stats, 'Tweets': td_stats}
    
    return pd.DataFrame(col_stats).T


calculate_collection_stats()

,Total,Congress,Republican
Handles,796,537,263
Users,785,535,261
Tweets,100078,75039,37128


#### PPTX13 - Model Performance Findings (Slide 13)

In [5]:
cp = {
    'AzureDark': '#021F33',
    'BrownDark': '#804419',
    'AzureFaded': '#466F91',
    'BrownFaded': '#916846',
    'AzureBg': '#CAD9E6',
    'BrownBg': '#E6D6CA'
    }

## prepare data
best_model_stats[('time', 'run_time')] = (best_model_stats[('time', 'run_time')
        ]).fillna(min(best_model_stats[('time', 'run_time')].values))
best_model_stats = best_model_stats.reset_index()

## drop staged random guess model
i = best_model_stats[('model','variant')] != 'stage'
best_model_stats = best_model_stats.loc[i]

## configure plotting parameters to represent different model
best_model_stats[('label', 'model')] = best_model_stats[(
    'model', 'algorithm')].replace({'bayes':'NB', 'forest':'RF',
    'logistic':'LR', 'adaboost':'AB', 'random':'RG'})

bm_col = {
    'feature_unitary': cp['BrownDark'],
    'feature_stage': cp['AzureFaded'],
    'pca_unitary': cp['BrownFaded'],
    'pca_stage': cp['AzureFaded'],
    'unitary': cp['BrownDark'],
    'stage': cp['AzureDark'],
}

best_model_bbox = {
    'feature_unitary': dict(boxstyle = 'round', linestyle = '-',
            facecolor = '#FFFFFF80', edgecolor = bm_col['feature_unitary'] ),
    'feature_stage': dict(boxstyle = 'round', linestyle = '-',
            facecolor = '#FFFFFF00', edgecolor = bm_col['feature_stage']),
    'pca_unitary': dict(boxstyle = 'round', linestyle = ':',
            facecolor = '#FFFFFF00', edgecolor = bm_col['pca_unitary']),
    'pca_stage': dict(boxstyle = 'round', linestyle = ':',
            facecolor = '#FFFFFF00', edgecolor = bm_col['pca_stage']),
    'unitary': dict(boxstyle = 'round', linestyle = '-',
            facecolor = '#FFFFFF80', edgecolor = bm_col['unitary']),
    'stage': dict(boxstyle = 'round', linestyle = '-',
            facecolor = '#FFFFFF00', edgecolor = bm_col['stage']),
    }

In [ ]:
matplotlib.rcParams['axes.edgecolor'] = cp['AzureDark']
matplotlib.rcParams['axes.labelcolor'] = cp['AzureDark']

## generate f1 x negative runtime performance plot
plt.scatter(
    x = best_model_stats[('time', 'run_time')],
    y = best_model_stats[('perform', 'f1')],
    c = '#FFFFFF00'
)

best_model = ['feature_unitary', 'unitary']

## render non-best model variants
for i in best_model_stats.index:
    if best_model_stats.loc[i, ('model', 'variant')] not in best_model:
        plt.text(
            x = best_model_stats.loc[i, ('time', 'run_time')],
            y = best_model_stats.loc[i, ('perform', 'f1')],
            s = best_model_stats.loc[i, ('label', 'model')],
            bbox = best_model_bbox[best_model_stats.loc[i, ('model', 'variant')]],
            c = bm_col[best_model_stats.loc[i, ('model', 'variant')]]
    )

## render best model variant
for i in best_model_stats.index:
    if best_model_stats.loc[i, ('model', 'variant')] in best_model:
        plt.text(
            x = best_model_stats.loc[i, ('time', 'run_time')],
            y = best_model_stats.loc[i, ('perform', 'f1')],
            s = best_model_stats.loc[i, ('label', 'model')],
            bbox = best_model_bbox[best_model_stats.loc[i, ('model', 'variant')]],
            c = bm_col[best_model_stats.loc[i, ('model', 'variant')]]
    )

import matplotlib.patches as mpatches
        
the_handles = [
    mpatches.Patch(edgecolor =  bm_col['feature_unitary'],
        label='One-Stage, Features', facecolor = '#FFFFFF80', linestyle = '-'),
    mpatches.Patch(edgecolor = bm_col['pca_unitary'], label='One-Stage, PCA',
        facecolor = cp['AzureBg'], linestyle = ':'),
    mpatches.Patch(edgecolor = bm_col['feature_stage'], label='Two-Stage, Features',
        facecolor = cp['AzureBg'], linestyle = '-'),
    mpatches.Patch(edgecolor=bm_col['pca_stage'], label='Two-Stage, PCA',
        facecolor = cp['AzureBg'], linestyle = ':')
]

plt.legend(handles = the_handles,
           facecolor = cp['AzureBg'], edgecolor = '#00000000')


## plot parameters
plt.gcf().set_size_inches(4.5, 4.5)
plt.gcf().set_facecolor(cp['AzureBg'])

plt.gca().set_facecolor(cp['AzureBg'])
plt.gca().set_xlabel('Seconds of runtime (logged) • Model is fast ---->')
plt.gca().set_ylabel('F1 score on test dataset • Model is accurate ---->')
plt.gca().set_xlim((0.95, 2**11))
plt.gca().set_ylim(0.3, 1)
plt.gca().set_xscale('log', base = 2)
plt.gca().invert_xaxis()
plt.gca().tick_params(axis='x', colors = cp['AzureDark'])
plt.gca().tick_params(axis='y', colors = cp['AzureDark'])


plt.gcf().savefig('C_Output/model_performance.pdf')

## FOOT - display objects as needed